In [17]:
import torch
import os
import numpy as np
import shutil

dataset_dir = os.path.join("D:\\Data\\University\\MOTSynth","mini-dataset-2")

training_ratio = 0.8 # (1 - training_ratio) will be the portion for tests
gallery_ratio = 0.9  # (1 - gallery_ratio) will be the portion for queries

In [32]:
def get_folders_from_folder(path):
    elements = os.listdir(path)
    dirs = [el for el in elements if os.path.isdir(os.path.join(path, el))]
    return np.asarray(dirs)

def get_files_from_folder(path):
    elements = os.listdir(path)
    return np.asarray(elements)    

dirs = get_folders_from_folder(dataset_dir)
np.random.shuffle(dirs)
n_folders = len(dirs)

train_dir = os.path.join(dataset_dir,"train")
os.mkdir(train_dir)
queries_dir = os.path.join(dataset_dir,"queries")
os.mkdir(queries_dir)
gallery_dir = os.path.join(dataset_dir,"gallery")
os.mkdir(gallery_dir)

train_set, test_set = np.split(dirs, [round(training_ratio*n_folders)])

#print("train: ",train_set, len(train_set))
#print("test: ", test_set, len(test_set))

for i in train_set:
    shutil.move(os.path.join(dataset_dir,i), os.path.join(train_dir,i))



FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:\\Data\\University\\MOTSynth\\mini-dataset-2\\train'

In [35]:
for i in test_set:
    src_dir = os.path.join(dataset_dir,i)
    query_dst = os.path.join(queries_dir,i)
    os.mkdir(query_dst)
    gallery_dst = os.path.join(gallery_dir,i)
    os.mkdir(gallery_dst)

    elements = get_files_from_folder(src_dir)
    np.random.shuffle(elements)
    n_elements = len(elements)
    idx = round(gallery_ratio * n_elements)
    idx = min(idx, n_elements-1)    # we want at least one query
    gallery, queries = np.split(elements, [idx])
    
    for j in gallery:
        shutil.move(os.path.join(src_dir,j), os.path.join(gallery_dst,j))

    for j in queries:
        shutil.move(os.path.join(src_dir,j), os.path.join(query_dst,j))
    os.rmdir(src_dir)
    print("Split ", i, " into gallery & queries")    
